In [1]:
import pandas as pd
dataset_kmeans = pd.read_csv('/Users/divyatmika/Downloads/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')

In [80]:
dataset_full = pd.read_csv('/Users/divyatmika/Downloads/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')

In [2]:
dataset_kmeans.shape

(7195, 26)

In [3]:
dataset_kmeans.drop(columns=['Family','Genus','Species'],inplace=True)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(dataset_kmeans)

# 2(a. ) Iterate 50 times and find best k for kmeans using Silhouette score

In [83]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np

best_k_res = {}
for iteration in range(1,51,1):
    scores = {}
    for n_clusters in range(2,20,2):
        random_state = 1
        clusterer = KMeans(n_clusters=n_clusters, random_state=random_state)
        cluster_labels = clusterer.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        scores.update({n_clusters:silhouette_avg})
        random_state = random_state + 5
        
    best_k = max(scores, key=scores.get)
    best_k_res.update({iteration:best_k})

# Overall best k = 4 

In [111]:
best_k_res

{1: 4,
 2: 4,
 3: 4,
 4: 4,
 5: 4,
 6: 4,
 7: 4,
 8: 2,
 9: 4,
 10: 4,
 11: 4,
 12: 3,
 13: 4,
 14: 4,
 15: 4,
 16: 4,
 17: 4,
 18: 4,
 19: 4,
 20: 5,
 21: 4,
 22: 3,
 23: 4,
 24: 4,
 25: 4,
 26: 3,
 27: 4,
 28: 4,
 29: 4,
 30: 5,
 31: 4,
 32: 4,
 33: 4,
 34: 3,
 35: 4,
 36: 3,
 37: 4,
 38: 4,
 39: 5,
 40: 5,
 41: 4,
 42: 4,
 43: 4,
 44: 4,
 45: 4,
 46: 4,
 47: 4,
 48: 4,
 49: 4,
 50: 4}

# 2(b.) In each cluster, determine which family, genus and species is majority : 50 times
1. Take the best k from previous question
2. loop over all the best k 
3. for each K :
   for each cluster : 
    calculate the majority of fam, genus, species by groupby clusters!
4. Get the triplet. 
5. Map the labeled data. And then calculate the hamming loss by taking difference from true and labels generated by clsuters 

In [119]:
#Train on best k 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np
hamming_loss_score = []
ans_cluster_iter = {}
for k,v in best_k_res.items():
    print("Iteration number : ",k)
    k_val = best_k_res.get(k)
    kmeans = KMeans(n_clusters=best_k_res.get(k))
    kmeans.fit(X)
    labels = kmeans.labels_
    #assign clusters in the dataset 
    dataset_kmeans['clusters'] = labels
    #concat the result 
    result = pd.concat([dataset_full, dataset_kmeans], axis=1, join="inner")
    #get for families:
    Y_family = result[['Family','clusters']]
    count_fam = Y_family.groupby(['clusters','Family']).size() #group by families to take majority polling 
    my_dict_fam = {}
    for i, v in count_fam.items():
        y = i
        if y[0] in my_dict_fam:
            if v > my_dict_fam.get(y[0]):
                my_dict_fam.update({y[0]:v})
        else:
            my_dict_fam.update({y[0]:v})
    answer_fam = []
    for key_dict, value_dict in my_dict_fam.items():
        series = count_fam.get(key_dict)
        for key_ser, val_ser in series.items():
            if my_dict_fam.get(key_dict) == val_ser:
                answer_fam.append(key_ser)
    #do same thing for genus 
    Y_genus = result[['Genus','clusters']]
    count_genus = Y_genus.groupby(['clusters','Genus']).size()
    my_dict_genus = {}
    for i, v in count_genus.items():
        y = i
        if y[0] in my_dict_genus:
            if v > my_dict_genus.get(y[0]):
                my_dict_genus.update({y[0]:v})
        else:
            my_dict_genus.update({y[0]:v})
    answer_genus = []
    for key_dict, value_dict in my_dict_genus.items():
        series = count_genus.get(key_dict)
        for key_ser, val_ser in series.items():
            if my_dict_genus.get(key_dict) == val_ser:
                answer_genus.append(key_ser)
    #for species : 
    Y_species = result[['Species','clusters']]
    count_species = Y_species.groupby(['clusters','Species']).size()
    my_dict_species = {}
    for i, v in count_species.items():
        y = i
        if y[0] in my_dict_species:
            if v > my_dict_species.get(y[0]):
                my_dict_species.update({y[0]:v})
        else:
            my_dict_species.update({y[0]:v})
    answer_species = []
    for key_dict, value_dict in my_dict_species.items():
        series = count_species.get(key_dict)
        for key_ser, val_ser in series.items():
            if my_dict_species.get(key_dict) == val_ser:
                answer_species.append(key_ser)
    ans_cluster = {}
    for i in range(0,k_val,1):
        triplet = []
        triplet.append(answer_fam[i])
        triplet.append(answer_genus[i])
        triplet.append(answer_species[i])
        ans_cluster.update({i:triplet})
    print("For each clsuter of k,this represents the majority of each label : ")
    print(ans_cluster)
    print("/n")
    #ans_cluster_iter.update({k:ans_cluster})
    for i in range(0,k_val,1):
        result['Family'][result.clusters == i] = answer_fam[i]
        result['Genus'][result.clusters == i] = answer_genus[i]
        result['Species'][result.clusters == i] = answer_species[i]
    #HAMMING SCORE CALCULATION : 
    fam_loss = hamming_loss(dataset_full['Family'],result['Family'])
    gen_loss = hamming_loss(dataset_full['Genus'],result['Genus'])
    species_loss = hamming_loss(dataset_full['Species'],result['Species'])
    hamming_loss_val = (fam_loss + gen_loss + species_loss)/3
    hamming_loss_score.append(hamming_loss_val)

Iteration number :  1
For each clsuter of k,this represents the majority of each label : 
{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 2: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 3: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre']}
/n
Iteration number :  2
For each clsuter of k,this represents the majority of each label : 
{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens']}
/n
Iteration number :  3
For each clsuter of k,this represents the majority of each label : 
{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre']}
/n
Iteration number :  4
For each clsuter of k,this re

For each clsuter of k,this represents the majority of each label : 
{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 2: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 3: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre']}
/n
Iteration number :  29
For each clsuter of k,this represents the majority of each label : 
{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre']}
/n
Iteration number :  30
For each clsuter of k,this represents the majority of each label : 
{0: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 4: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre']}
/n
Iteration number :

# The above result shows that for each iteration : now for each k say 4 : we have 4 clusters : so it shows that for each cluster of k, (i.e. 0,1,2,3): it shows the majority for family, genus and species.

# Hamming Loss : 

# After mapping the data : compare the true lables and the lables done by the clusters and now one can take the average of misclassifications done for genus and family and species and get the final hamming score: 
Code is above : 

In [137]:
hamming_loss_score

[0.777577,
 0.777577,
 0.777577,
 0.777531,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.7754876,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.7754876,
 0.7754889,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.754786,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577,
 0.777577]

In [133]:
hamming_score_result = pd.DataFrame(hamming_loss_score)

# Hamming score Mean :

In [134]:
hamming_score_result.mean()

0    0.776995
dtype: float64

# Hamming score Stddev : 

In [136]:
hamming_score_result.std()

0    0.003244
dtype: float64

# Hamming loss is approx : 0.23